In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision import models
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import seaborn as sns
import pandas as pd
from collections import OrderedDict
import warnings
warnings.filterwarnings("ignore")

# ===============================================
# 1. ГЛОБАЛЬНЫЕ НАСТРОЙКИ (гиперпараметры)
# ===============================================
DATA_DIR       = 'path/to/your/dataset'          # Основная папка с данными
TRAIN_DIR      = os.path.join(DATA_DIR, 'train')  # Папка с тренировочными изображениями
VAL_DIR        = os.path.join(DATA_DIR, 'val')    # Папка с валидационными изображениями
TEST_DIR       = os.path.join(DATA_DIR, 'test')   # Папка с тестовыми изображениями (может быть без меток или с ними)

NUM_CLASSES    = 10                    # <<< ИЗМЕНИТЕ НА СВОЁ КОЛИЧЕСТВО КЛАССОВ
BATCH_SIZE     = 64
NUM_EPOCHS     = 50
LEARNING_RATE  = 0.001
WEIGHT_DECAY   = 1e-4                  # L2-регуляризация
DEVICE         = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
PRINT_FREQ     = 100                   # Как часто выводить loss во время эпохи
SEED           = 42                    # Для воспроизводимости

# Фиксируем случайность — важный момент для экспериментов!
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
np.random.seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

print(f"Используем устройство: {DEVICE}")
print(f"Количество классов: {NUM_CLASSES}")

# ===============================================
# 2. АУГМЕНТАЦИИ И ЗАГРУЗКА ДАННЫХ
# ===============================================

# Тренировочные аугментации — делаем данные разнообразнее, чтобы модель лучше обобщала
transform_train = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),  # Случайный кроп с изменением масштаба
    transforms.RandomHorizontalFlip(p=0.5),              # Горизонтальный поворот
    transforms.RandomRotation(15),                       # Поворот до ±15°
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomGrayscale(p=0.1),                   # Иногда делаем ч/б
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],      # Статистика ImageNet
                         std =[0.229, 0.224, 0.225]),
])

# Валидация и тест — только детерминированные трансформации (без случайности)
transform_val_test = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std =[0.229, 0.224, 0.225]),
])

# Загружаем датасеты через ImageFolder (структура: папка_класса/изображения)
train_dataset = ImageFolder(TRAIN_DIR, transform=transform_train)
val_dataset   = ImageFolder(VAL_DIR,   transform=transform_val_test)

# Тестовый датасет может быть:
#   а) с метками (тогда тоже ImageFolder)
#   б) без меток — тогда используем torchvision.datasets.ImageFolder с кастомным loader'ом (ниже показано оба варианта)
test_dataset = ImageFolder(TEST_DIR, transform=transform_val_test)  # если есть метки

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True,
                          num_workers=8, pin_memory=True, drop_last=False)
val_loader   = DataLoader(val_dataset,   batch_size=BATCH_SIZE, shuffle=False,
                          num_workers=8, pin_memory=True)
test_loader  = DataLoader(test_dataset,  batch_size=BATCH_SIZE, shuffle=False,
                          num_workers=8, pin_memory=True)

# Названия классов (очень полезно для отчётов)
class_names = train_dataset.classes
class_to_idx = train_dataset.class_to_idx
print(f"Классы ({len(class_names)}): {class_names}")
print(f"class_to_idx: {class_to_idx}")

# ===============================================
# 3. СОЗДАНИЕ МОДЕЛИ (Transfer Learning)
# ===============================================

# Выбираем любую предобученную модель. Здесь — ResNet50 (можно заменить на EfficientNet, Swin и т.д.)
base_model = models.resnet50(pretrained=True)  # weights=models.ResNet50_Weights.IMAGENET1K_V2 в новых версиях

# --- Вариант A: Замораживаем все слои кроме последнего (быстрее, меньше переобучения) ---
for param in base_model.parameters():
    param.requires_grad = False

# Меняем финальный слой под наши классы
num_features = base_model.fc.in_features
base_model.fc = nn.Linear(num_features, NUM_CLASSES)

# Если хотите дообучать всю сеть (fine-tuning) — раскомментируйте:
# for param in base_model.parameters():
#     param.requires_grad = True

model = base_model.to(DEVICE)

# ===============================================
# 4. ФУНКЦИЯ ПОТЕРЬ, ОПТИМИЗАТОР, ПЛАНИРОВЩИК
# ===============================================
criterion = nn.CrossEntropyLoss()  # Для многоклассовой классификации

# Оптимизатор только по тем параметрам, которые обучаются
optimizer = optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()),
                        lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)

# Планировщик обучения — косинусный отжиг (очень эффективен)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=NUM_EPOCHS)

# ===============================================
# 5. ВСПОМОГАТЕЛЬНЫЕ ФУНКЦИИ ОБУЧЕНИЯ И ВАЛИДАЦИИ
# ===============================================

def train_one_epoch(epoch_idx):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    pbar = tqdm(train_loader, desc=f'Epoch {epoch_idx+1:02d}/{NUM_EPOCHS} [TRAIN]')
    for batch_idx, (images, targets) in enumerate(pbar):
        images = images.to(DEVICE, non_blocking=True)
        targets = targets.to(DEVICE, non_blocking=True)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        # Обновляем прогресс-бар
        if batch_idx % PRINT_FREQ == 0 or batch_idx == len(train_loader)-1:
            pbar.set_postfix({
                'loss': running_loss / (batch_idx + 1),
                'acc' : 100. * correct / total,
                'lr'  : optimizer.param_groups[0]['lr']
            })

    epoch_loss = running_loss / len(train_loader)
    epoch_acc  = 100. * correct / total
    return epoch_loss, epoch_acc


@torch.no_grad()
def validate(epoch_idx):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    all_preds = []
    all_labels = []

    pbar = tqdm(val_loader, desc=f'Epoch {epoch_idx+1:02d}/{NUM_EPOCHS} [VAL  ]', leave=False)
    for images, targets in pbar:
        images = images.to(DEVICE, non_blocking=True)
        targets = targets.to(DEVICE, non_blocking=True)

        outputs = model(images)
        loss = criterion(outputs, targets)

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(targets.cpu().numpy())

    epoch_loss = running_loss / len(val_loader)
    epoch_acc  = 100. * correct / total
    print(f"VALIDATION → Loss: {epoch_loss:.4f} | Accuracy: {epoch_acc:.2f}%")
    return epoch_loss, epoch_acc, np.array(all_preds), np.array(all_labels)


# ===============================================
# 6. ОСНОВНОЙ ЦИКЛ ОБУЧЕНИЯ
# ===============================================
best_val_acc = 0.0
history = {
    'train_loss': [], 'train_acc': [],
    'val_loss'  : [], 'val_acc'  : []
}

print("\n=== НАЧИНАЕМ ОБУЧЕНИЕ ===\n")
for epoch in range(NUM_EPOCHS):
    train_loss, train_acc = train_one_epoch(epoch)
    val_loss,   val_acc, _, _ = validate(epoch)
    
    scheduler.step()  # обновляем LR

    # Сохраняем историю
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)

    # Сохраняем лучшую модель по валидационной точности
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save({
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_acc': val_acc
        }, 'best_model.pth')
        print(f"  → НОВАЯ ЛУЧШАЯ МОДЕЛЬ! Val Acc: {best_val_acc:.2f}%\n")

print(f"\nОБУЧЕНИЕ ЗАВЕРШЕНО. Лучшая валидационная точность: {best_val_acc:.2f}%\n")

# ===============================================
# 7. ФИНАЛЬНАЯ ОЦЕНКА НА ВАЛИДАЦИИ + МЕТРИКИ
# ===============================================
print("=== ЗАГРУЖАЕМ ЛУЧШУЮ МОДЕЛЬ ДЛЯ ФИНАЛЬНОЙ ОЦЕНКИ ===")
checkpoint = torch.load('best_model.pth', map_location=DEVICE)
model.load_state_dict(checkpoint['model_state_dict'])

_, _, val_preds, val_labels = validate(0)  # просто прогоняем ещё раз

print("\n" + "="*50)
print("CLASSIFICATION REPORT (VALIDATION)")
print("="*50)
print(classification_report(val_labels, val_preds, target_names=class_names, digits=4))

# Confusion Matrix
cm = confusion_matrix(val_labels, val_preds)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix (Validation)')
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

# ===============================================
# 8. ПРЕДСКАЗАНИЕ НА ТЕСТОВОМ НАБОРЕ (TEST SET)
# ===============================================
@torch.no_grad()
def predict_test(loader, save_csv=True):
    model.eval()
    all_filenames = []
    all_preds = []
    all_probs = []
    all_labels = [] if hasattr(loader.dataset, 'targets') else None

    print("\n=== ПРЕДСКАЗАНИЕ НА ТЕСТОВОМ НАБОРЕ ===")
    pbar = tqdm(loader, desc="Test inference")
    
    for images, targets_or_paths in pbar:
        # ImageFolder возвращает targets, но если вы используете кастомный Dataset — можно передавать пути
        if isinstance(targets_or_paths, tuple):  # если у вас кастомный loader с путями
            images, paths = images
        else:
            paths = None

        images = images.to(DEVICE, non_blocking=True)

        outputs = model(images)
        probabilities = torch.softmax(outputs, dim=1)
        _, predicted = outputs.max(1)

        # Сохраняем имена файлов (если доступны)
        if paths is not None:
            all_filenames.extend(paths)
        elif hasattr(loader.dataset, 'imgs'):
            # ImageFolder хранит пути в .imgs
            batch_start = len(all_filenames)
            batch_paths = [loader.dataset.imgs[i][0] for i in range(batch_start, batch_start + images.size(0))]
            all_filenames.extend([os.path.basename(p) for p in batch_paths])

        all_preds.extend(predicted.cpu().numpy())
        all_probs.extend(probabilities.cpu().numpy())
        if all_labels is not None:
            all_labels.extend(targets_or_paths.cpu().numpy() if torch.is_tensor(targets_or_paths) else targets_or_paths)

    # Преобразуем предсказания в названия классов
    pred_labels = [class_names[idx] for idx in all_preds]
    prob_df = pd.DataFrame(all_probs, columns=[f'prob_{c}' for c in class_names])

    results_df = pd.DataFrame({
        'filename': all_filenames if all_filenames else [f'img_{i}' for i in range(len(all_preds))],
        'predicted_class_id': all_preds,
        'predicted_class': pred_labels,
        'confidence': prob_df.max(axis=1).values  # максимальная вероятность
    })
    results_df = pd.concat([results_df, prob_df], axis=1)

    if save_csv:
        results_df.to_csv('test_predictions.csv', index=False)
        print(f"Предсказания сохранены в 'test_predictions.csv' ({len(results_df)} записей)")

    # Если в тесте есть метки — выводим точность
    if all_labels is not None and len(all_labels) > 0:
        test_acc = accuracy_score(all_labels, all_preds) * 100
        print(f"Test Accuracy: {test_acc:.2f}%")

    return results_df

# Запускаем предсказание
test_results_df = predict_test(test_loader, save_csv=True)

# Показываем первые 10 строк результата
print("\nПервые 10 предсказаний:")
print(test_results_df.head(10))

# ===============================================
# 9. ГРАФИКИ ОБУЧЕНИЯ
# ===============================================
plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
plt.plot(history['train_loss'], label='Train Loss')
plt.plot(history['val_loss'],   label='Val Loss')
plt.title('Loss during training')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(history['train_acc'], label='Train Acc')
plt.plot(history['val_acc'],   label='Val Acc')
plt.title('Accuracy during training')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()